<a href="https://colab.research.google.com/github/fundaylncii/NaturalLanguageProcessing/blob/main/NLPSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
from wordcloud import WordCloud

In [52]:
filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width",200)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [53]:
import nltk
nltk.download("stopwords")
sw = stopwords.words("english")

nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [54]:
df = pd.read_csv("/content/amazon_reviews.csv")

In [55]:
## TextPreprocessing işleminin fonksiyonlaştırılması

def preprocess_text(text):
  ## metinleri küçük harfe çevirme
  text = text.str.lower()

  ## metin içerisinden noktalama işaretlerinin kaldırılması
  text = text.str.replace("[^\w\s]", "",regex=True)

  ## metin içerisinden sayısal değerlerin kaldırılması
  text = text.str.replace("\d","",regex=True)

  ## metin içerisinden örüntü oluşturmayacak sık sık kullanılan terimlerin çıkartılması (stopwords)
  text = text.apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

  ## metin içerisinde sıklık referansı 1 ve küçük olanların çıkartılması
  temp_df = pd.Series(" ".join(text).split()).value_counts()
  drop = temp_df[temp_df <= 1]
  text = text.apply(lambda x: " ".join(x for x in str(x).split() if x not in drop))

  ## metin içeriisndeki kelimeleri köklerine ayırma (stemming)
  text = text.apply(lambda x: " ".join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))

  return text

In [56]:
## reviewText alanının TextPreprocessing işlemine tabi tutma

df["reviewText"] = preprocess_text(df["reviewText"])
df["reviewText"]

,reviewText
0,issue
1,purchased device worked advertised never much ...
2,work expected higher capacity think made bit e...
3,think worked gb card went south one held prett...
4,bought retail packaging arrived legit envelope...
...,...
4910,bought sandisk gb class use htc inspire month ...
4911,used capability samsung galaxy note greatly ex...
4912,great card fast reliable come optional adapter...
4913,good amount space stuff want fit gopro say


In [57]:
## Bir metnin pozitif - negatif olması belirlenir
df["reviewText"].head()

,reviewText
0,issue
1,purchased device worked advertised never much ...
2,work expected higher capacity think made bit e...
3,think worked gb card went south one held prett...
4,bought retail packaging arrived legit envelope...


In [58]:
## kelimelerin tek başına taşıdığı anlamlar kontrol ediler
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [59]:
## polarity score : negatif- pozitif olma skoru

sia = SentimentIntensityAnalyzer()

In [60]:
## örnek metin
sia.polarity_scores("The film was awesome")
## compound > 0 olduğu için metin pozitif anlam taşıyor

{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.6249}

In [11]:
## duygu durumu skorlar -1 +1 arasında değer alır ve polarity_scores işleminde compound değeri önemlidir
## compound > 0 ise metin pozitif anlam taşıyor
## compound < 0 ise metin negatif anlam taşıyor

In [61]:
sia.polarity_scores("I liked this music but it is not good as the other one")
## compound < 0 olduğu için metin negatif anlam taşıyor

{'neg': 0.207, 'neu': 0.666, 'pos': 0.127, 'compound': -0.298}

In [62]:
## reviewText ler içerisinde her bir satır için skoreların elde edilmesi

df["reviewText"][0:10].apply(lambda x: sia.polarity_scores(x)["compound"])

,reviewText
0,0.00
1,0.00
2,0.40
3,0.65
4,0.86
5,0.00
6,0.87
7,0.82
8,0.00
9,0.92


In [63]:
## df içerisine polarity scorenun eklenmesi

df["polarity_score"] = df["reviewText"].apply(lambda x: sia.polarity_scores(x)["compound"])

In [64]:
df.head()
## polarity_score == 0 ise nötr yorum

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,polarity_score
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",issue,4.00,Four Stars,1406073600,2014-07-23,138,0,0,0.00
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]",purchased device worked advertised never much ...,5.00,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0.00
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",work expected higher capacity think made bit e...,4.00,nothing to really say....,1356220800,2012-12-23,715,0,0,0.40
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",think worked gb card went south one held prett...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0.65
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]",bought retail packaging arrived legit envelope...,5.00,best deal around,1373673600,2013-07-13,513,0,0,0.86


In [ ]:
## dataset içerisindeki ürün puanları ile polarity scoreu değerlendirilmelidir
## overall değeri 3 ve daha az puan olan ancak polarity score değeri 0 dan büyük değerlendirmeler yanlış değerlendirmelerdir

In [65]:
## overall < 3 ve polarity_score > 0 olanlar

df[(df["overall"]<3) & (df["polarity_score"] > 0)].head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,polarity_score
38,A1FKE13D77L3Y3,B007WTAJTO,Abraham Arturo Meza Marin,"[0, 0]",bougth micro sd card got kingston tell sandisk...,1.00,Stop working after 1 year,1361232000,2013-02-19,657,0,0,0.94
108,A28CTDM7OP0RAR,B007WTAJTO,Alan,"[0, 0]",work actual rw speed mb even class sandisk bas...,2.00,Painfully slow,1395705600,2014-03-25,258,0,0,0.42
123,A1UM44ILLZCEI1,B007WTAJTO,Aleksandar Milivojevic,"[1, 1]",bought gb version card use gopro hero black ed...,2.00,Didn't work out for me,1364947200,2013-03-04,644,1,1,0.94
150,A87U7LY9OKZNY,B007WTAJTO,A. Liu,"[0, 0]",used galaxy note past month sudden longer trie...,2.00,Not writable after 16 months,1398988800,2014-02-05,306,0,0,0.62
177,A2VBZVFBSIOMS3,B007WTAJTO,"Amazon Customer ""Christian""","[0, 0]",use sdxc card gb card unreadable arrival pc sa...,1.00,Dead on arrival,1369180800,2013-05-22,565,0,0,0.77


In [66]:
## overall == 5 ve polarity_score < 0

df[(df["overall"] == 5) & (df["polarity_score"] < 0)].head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,polarity_score
19,A2ELBSIZ26GKM2,B007WTAJTO,Aaron,"[0, 0]",work video stuttering like gb blah blah blah b...,5.00,works,1389052800,2014-07-01,160,0,0,-0.34
20,A6AL9BZ0JALUP,B007WTAJTO,Aaron,"[0, 0]",work expected high transfer speed nice extra m...,5.00,Works,1382832000,2013-10-27,407,0,0,-0.15
24,A243XIFG20QRQW,B007WTAJTO,Aaron Nash,"[0, 0]",san disk hard beat pay brand get quality produ...,5.00,Perfect,1395532800,2014-03-23,260,0,0,-0.20
34,A747BB4QM2SWW,B007WTAJTO,A. Bell,"[0, 0]",ive got couple varying size ive problem compla...,5.00,several sizes. zero issues.,1376784000,2013-08-18,477,0,0,-0.72
36,A2OP1LOKO6P5W4,B007WTAJTO,Abe The Babe,"[0, 0]",bought surface pro ive month ive problem fast ...,5.00,Works great!,1375142400,2013-07-30,496,0,0,-0.13


In [67]:
## Sentiment Modeling

## geliştirilecek modeli ile modele sorulan yorumun pozitif negatif olduğunun belirlenmesi
## amaç : unsupervised problemi supervised probleme çevirme - sınıflandırma probleminin çözümlenmesi

## unsupervised learning den supervised learninge geçiş işlemi : müşteri segmentleri belirleniyor. k-means ile mevcut müşterilerin 3 sınıfı ayrıldığı varsayılsın(unsupervised learning ile bulunuyor sınıflar)
## sınıflar dataset içerisine label olarak eklenerek problem sınıflandırma problemine dönüştürülür. (supervised learninge geçiş yapıyor)
## yeni müşteri geldiğinde bu müşterinin hangi sınıfa dahil olacağı bulunmak isteniyor.(supervised learning kullanılarak bulunur)

## bizim işlemlerimizde metinlerdeki polarity_scoreunun bulunması unspervised learning ile  tespit edildi.
## bu scorelara göre label eklenip negatif - pozitif değer ataması yaparak bunu spervised learninge çevireceğiz.
## yeni bir metin geldiğinde bunun negatif -  pozitif olduğunun bulunması ise bir supervised problem olmuş olacak

## step 1 : feature engineering

## labelin oluşturulması

df["sentiment_label"] = df["reviewText"].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,polarity_score,sentiment_label
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",issue,4.00,Four Stars,1406073600,2014-07-23,138,0,0,0.00,neg
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]",purchased device worked advertised never much ...,5.00,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0.00,neg
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",work expected higher capacity think made bit e...,4.00,nothing to really say....,1356220800,2012-12-23,715,0,0,0.40,pos
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",think worked gb card went south one held prett...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0.65,pos
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]",bought retail packaging arrived legit envelope...,5.00,best deal around,1373673600,2013-07-13,513,0,0,0.86,pos


In [68]:
df["sentiment_label"].value_counts()

,count
sentiment_label,
pos,3944
neg,971


In [69]:
## pozitif-negatif olanların puan ortalaması

df.groupby("sentiment_label")["overall"].mean()

,overall
sentiment_label,
neg,4.09
pos,4.71


In [70]:
## sentiment_label değerinin encode edilmesi gereklidi

df["sentiment_label"] = LabelEncoder().fit_transform(df["sentiment_label"])
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,polarity_score,sentiment_label
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",issue,4.00,Four Stars,1406073600,2014-07-23,138,0,0,0.00,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]",purchased device worked advertised never much ...,5.00,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0.00,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",work expected higher capacity think made bit e...,4.00,nothing to really say....,1356220800,2012-12-23,715,0,0,0.40,1
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",think worked gb card went south one held prett...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0.65,1
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]",bought retail packaging arrived legit envelope...,5.00,best deal around,1373673600,2013-07-13,513,0,0,0.86,1


In [71]:
# taget / bagımlı değişken
y = df["sentiment_label"]
X = df["reviewText"]

In [23]:
## bağımsız değişkenimiz df["reviewText"] bir metin ve ölçümlenebilir bir değer değil.

## COUNT VECTORS : Frekansları temsil eder
## TF-IDF VECTORS: normalize edilmiş freaknasları temsil eder
## WORD EMBEDDINGS (WORD2VEC, GLOVE, BERT VS)

In [24]:
## count vectors işlemi kelimelerin frekanslarının çıkarılması işlemidir

## words
## kelimelerin nümerik temsilleri


## characters
## karakterlerin nümerik temsilleri

## n-gram
## n- gramlar birlikte kullanılan kelimelerin kombinasyonlarını gösterir ve fature üretmek için kullanılır.

In [72]:
## count vectors
from sklearn.feature_extraction.text import CountVectorizer

In [73]:
corpus = ["This is the firs document.",
          "This document is the second document.",
          "And this is the third one.",
          "Is this the first document?"]

In [74]:
## word frekans
vectorizer = CountVectorizer()
X_c = vectorizer.fit_transform(corpus)

## corpusun içerisindeki eşsiz kelimeler çıkartılır
print(vectorizer.get_feature_names_out())
## kelimeler nümerik yapıya dönüştürülür.
X_c.toarray()

## her kelimenin text içerisindeki frekanslarını sayarak çıktı elde edilir.
## tüm satırlar tek bir metin gibi düşünülür. Metin içerisindeki eşsiz kelimeler çıkartılır ve eşsiz kelimelerin her bir metin içerisindeki frekanslarına göre bir array oluşturulur.

['and' 'document' 'firs' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


array([[0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 0, 1, 0, 1]])

In [75]:
## n-gram frekans

vectorizer2 = CountVectorizer(analyzer="word", ngram_range=(2,2 ))
X_n = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names_out())
X_n.toarray()

['and this' 'document is' 'firs document' 'first document' 'is the'
 'is this' 'second document' 'the firs' 'the first' 'the second'
 'the third' 'third one' 'this document' 'this is' 'this the']


array([[0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]])

In [76]:
## X = df["reviewText"] için count vectors uygulama

vectorizer = CountVectorizer()
X_count = vectorizer.fit_transform(X)
print(vectorizer.get_feature_names_out()[0:10])
X_count.toarray()

['ability' 'able' 'absolute' 'absolutely' 'abuse' 'accept' 'acceptable'
 'accepted' 'accepting' 'accepts']


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
## TF - IDF

## standartlaştırılmış kelime vektörü oluşturma işlemidir.
## term-frequency - ınverse document frequency

## 1-)count vektorizerin hesaplanması 2-) TF-Term Frequency hesaplama 3-) IDF - Inverse Document Frequency hesaplanması

## count vektorizer : kelimelerin her dökümandaki frekansı
## TF-Term Frequency : t teriminin ilgili dokümandaki frekansı / dokümandaki toplam terim sayısı
## IDF -Inverse Document Frequency : 1 + LOGE((TOPLAM DÖKUMAN SAYISI+1) / (İÇİNDE T TERİMİ OLAN DÖKUMAN SAYISI + 1))
## TF * IDF hesaplanır
## l2 normalizasyonu yapılır : satırların kareleri toplamının karakökünü bul, ilgili satırdaki tüm hücreleri bulduğun değere böl

In [31]:
## Count vertorize yöntemi tek başına kullanıldığında sınıf özelinde yanlılıklar oluşturabilir. Bu nedenle bu yanlılıkları ortadan kaldırmak için tf-idf kullanılır.
## tf-idf de count vectorize kullanarak değerlerin tüm metin içerisindeki ağırlıklarını bulur ve bunu normalize eder.

In [77]:
## UYGULAMA:

## word
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
tf_idf_word_vectorizer = TfidfVectorizer()
X_tf_idf_word = tf_idf_word_vectorizer.fit_transform(X)

In [34]:
## n-gram
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2,3))
X_tf_idf_ngram = tf_idf_ngram_vectorizer.fit_transform(X)

In [79]:
## SENTIMENT MODELING

## LOGISTIC REGRESSION

log_model = LogisticRegression().fit(X_tf_idf_word,y)

In [80]:
cross_val_score(log_model,
                X_tf_idf_word,
                y,
                scoring="accuracy",
                cv=5).mean()

0.830111902339776

In [81]:
new_review = pd.Series("this product is good")

In [82]:
## yeni metni sayısallaştırıp modele sormak lazım

new_review = TfidfVectorizer().fit(X).transform(new_review)

In [83]:
log_model.predict(new_review)

array([1])

In [84]:
## RONDOM FORESTS

## Count Vectors:
rf_model = RandomForestClassifier().fit(X_count,y)
cross_val_score(rf_model, X_count,y, cv=5, n_jobs=-1).mean()

0.8427263479145474

In [85]:
## TF-IDF WORD-LEVEL:

rf_model = RandomForestClassifier().fit(X_tf_idf_word,y)
cross_val_score(rf_model, X_count,y, cv=5, n_jobs=-1).mean()

0.8390640895218718

In [86]:
## TF-IDF N-GRAM:

rf_model = RandomForestClassifier().fit(X_tf_idf_ngram,y)
cross_val_score(rf_model, X_count,y, cv=5, n_jobs=-1).mean()

0.8419125127161751

In [87]:
## HİPERPARAMETRE OPTİMİZASYONU:
rf_model = RandomForestClassifier(random_state=17)
rf_params = {"max_depth": [8,None],
             "max_features":[7,"auto"],
             "min_samples_split":[2,5,8],
             "n_estimators":[100,200]}


In [88]:
rf_best_grid = GridSearchCV(rf_model,
                            rf_params,
                            cv=5,
                            n_jobs=-1,
                            verbose=1).fit(X_count,y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [89]:
rf_best_grid.best_params_

{'max_depth': None,
 'max_features': 7,
 'min_samples_split': 2,
 'n_estimators': 100}

In [90]:
rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X_count,y)

In [91]:
cross_val_score(rf_final, X_count,y, cv=5, n_jobs=-1).mean()

0.8128179043743643